# Read the dataset

There are 4 datasets to read:
- Customer demographic: cd
- customer address: ca
- transactions: ts
- new customer: nc

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
%matplotlib inline

In [ ]:
cd = pd.read_excel("KPMG_VI_New_raw_data_update_final.xlsx", sheet_name = 'CustomerDemographic', skiprows =1)
ca = pd.read_excel("KPMG_VI_New_raw_data_update_final.xlsx", sheet_name = 4, skiprows =1)
ts = pd.read_excel("KPMG_VI_New_raw_data_update_final.xlsx", sheet_name = 1, skiprows =1)
nc = pd.read_excel("KPMG_VI_New_raw_data_update_final.xlsx", sheet_name = 2, skiprows =1)

join cd,ca,ts by customer_id 

In [ ]:
#print(cd.info(), ca.info(), ts.info())

In [ ]:
#print(set(ca.customer_id)- set(cd.customer_id))
#print(set(cd.customer_id)- set(ca.customer_id))

In [ ]:
temp1 = pd.merge(ca,cd, how = 'outer', on = 'customer_id')

In [ ]:
df = pd.merge(ts,temp1, how ='outer', on = 'customer_id')

In [ ]:
#df.info()

# Data Quality Assessment

we will dive in each data to see what data it has and assess its consistency

In [ ]:
df.info()

In [ ]:
df.columns

transaction_id, product_id, customer_id is the system way to store customer, we just need to check the value_counts and the nan values

for transaction_date and DOB, since both of them are datetime, we will plot to check outliers or any abnormal time line

In [ ]:
rs = df.groupby('transaction_date').transaction_id.count()
#rs.index, rs.values
sns.lineplot(x = rs.index, y = rs.values)

In [ ]:
rs = df.groupby('DOB').transaction_id.count()
#rs.index, rs.values
sns.lineplot(x = rs.index, y = rs.values)


Since we find out that there are something strang in DOB, as in somepeople would have died now if they were born in 1840. Here we will dive in 

In [ ]:
df.DOB.describe()

In [ ]:
df['age'] = 2020 - df.DOB.apply(lambda x: int(x.strftime('%Y')) if x==x else None)

In [ ]:
sns.boxplot(df.age)

In [ ]:
df[['online_order','order_status']].apply(lambda x: x.value_counts(dropna=0))

In [ ]:
df.iloc[:,[6,7,8,9]].apply(lambda x: x.value_counts(dropna=0))

In [ ]:
pd.melt(df.loc[:,['list_price','standard_cost']])

In [ ]:
sns.boxplot(x ='variable', y = 'value', data =pd.melt(df.loc[:,['list_price','standard_cost']]))

In [ ]:
from datetime import datetime


In [ ]:
df.product_first_sold_date[0]


In [ ]:
df.product_first_sold_date =pd.to_timedelta(df.product_first_sold_date, unit = 'D')+ pd.to_datetime('1899-12-30')

In [ ]:
sns.lineplot(x = df.product_first_sold_date.value_counts().index, y = df.product_first_sold_date.value_counts().values)

In [ ]:
df[['country','state','gender']].apply(lambda x: set(x))


In [ ]:
df.state = df.state.replace(to_replace ={'New South Wales': 'NSW', 'Victoria':"VIC"})
df.gender = df.gender.replace(to_replace ={'Female': 'F', 'Femal':"F", "Male":"M"})

In [ ]:
sns.boxplot(df.past_3_years_bike_related_purchases)

In [ ]:
df[['job_industry_category']].apply(lambda x: set(x))

In [ ]:
df[['job_industry_category']].apply(lambda x: x.value_counts(dropna=0))

In [ ]:
print(df.wealth_segment.value_counts(dropna=0),df.deceased_indicator.value_counts(dropna=0))

In [ ]:
df.default

In [ ]:
df.owns_car.value_counts(dropna= 0)

In [ ]:
sns.boxplot(df.tenure)

In [ ]:
dt = pd.DataFrame(list(zip(df.columns,df.isna().sum()/20510*100, df.isna().sum())))
dt

In [ ]:
#df[df.transaction_id.notna()]

In [ ]:
20000-197-30

Drop rows that has na value & drop columns last_name and default as it is not really necessary


In [ ]:
df.dropna(subset =['transaction_id','brand','address','DOB','online_order'],axis =0,inplace = True)
df.drop(['last_name','default'], axis =1 , inplace = True)

In [ ]:
df.info()

# Machine Model

Now it is good enough to build something, we will drop a few more unnecessary columns before apply any algorithm. Dropped columns are : first_name, address, country, DOB, postcode, job_title

In [ ]:
dt = df.drop(['first_name', 'address', 'country', 'DOB', 'postcode', 'job_title'], axis = 1)
dt.info()

In [ ]:
df2 =dt.drop(['online_order','order_status','brand','product_line','product_class',
              'product_size','product_first_sold_date'], axis =1)

In [ ]:
revenue=df2[['transaction_id','customer_id','list_price']].groupby('customer_id').list_price.sum()
quantity = df2[['transaction_id','customer_id','list_price']].groupby('customer_id').transaction_id.count()

In [ ]:
revenue

In [ ]:
data = df2.drop(['transaction_id','product_id','list_price','standard_cost','transaction_date'], axis =1)

In [ ]:
data.drop_duplicates(inplace = True)


In [ ]:
temp = pd.merge(data,revenue, how = 'left', on = 'customer_id')
data = pd.merge(temp,quantity, how = 'left', on = 'customer_id')

In [ ]:
data.rename(columns ={'list_price':'revenue','transaction_id': 'quantity'}, inplace= True)

In [ ]:
data.set_index('customer_id')

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
mapping = {"N":0,"Y":1,"No":0,"Yes":1}
map2 = {'Mass Customer':0, 'Affluent Customer':1, 'High Net Worth':2}

In [ ]:
data.job_industry_category.fillna('unknown',inplace= True)

In [ ]:
data.deceased_indicator=data.deceased_indicator.map(mapping)
data.owns_car=data.owns_car.map(mapping)
data.wealth_segment= data.wealth_segment.map(map2)

In [ ]:
print(set(data.job_industry_category),set(data.wealth_segment))

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
data.job_industry_category = le.fit_transform(data.job_industry_category)
data.state = le.fit_transform(data.state)
data.gender= le.fit_transform(data.gender)

In [ ]:
data.set_index('customer_id',inplace= True)

# Apply clustering technique with kmeans

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
%matplotlib inline

In [ ]:
y_km = KMeans(n_clusters=5).fit(data)

In [ ]:
sum_of_squared_distance = []
K = range(2,10)
for k in K:
    km = KMeans(n_clusters =k)
    model = km.fit(data)
    ssq= model.inertia_
    sum_of_squared_distance.append(ssq)


In [ ]:
sns.lineplot(x = K, y = sum_of_squared_distance)

In [ ]:
y_km.labels_

In [ ]:
data['kmg'] = y_km.labels_

In [ ]:
data.groupby('kmg').quantity.count()

In [ ]:
data.groupby('kmg').revenue.mean()

In [ ]:
data.groupby('kmg').revenue.sum()

In [ ]:
data.groupby('kmg').revenue.mean()

In [ ]:
data[data.kmg==3].describe()

In [ ]:
data[data.kmg==4].describe()

In [ ]:
plt.bar(data.kmg, data.revenue)

In [ ]:
data

In [ ]:
from sklearn.cluster import AgglomerativeClustering
ac = AgglomerativeClustering(n_clusters =3, affinity = 'euclidean', linkage = 'complete')

In [ ]:
labels = ac.fit_predict(X)

# Apply Linear Regression

Split the data set

In [ ]:
X= data.drop(['revenue'],axis =1)
y = data.revenue

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)
y_hat = lm.predict(X_test)

In [ ]:
from sklearn import metrics

In [ ]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_hat)))